<a href="https://colab.research.google.com/github/Freaker99/car-detection/blob/main/car_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
Mount drive with data.

In [ ]:
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive')
root = '/content/drive/My Drive/car-detection/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install YOLOv5 package.

In [ ]:
!pip install yolov5

Install libriaries, clone GitHub repository, YOLOv5's weeights, install requirements.

In [ ]:
#!pip install opencv-python
#!pip install torch torchvision
#!pip install numpy
#!pip install ultralytics

#!git clone https://github.com/ultralytics/yolov5
#!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
#%cd yolov5
#!pip install -r requirements.txt

Import all dependencies.

In [ ]:
import os
import json
import glob
import shutil
import subprocess
import torch
import torch.optim as optim

from torchvision.transforms import transforms
from torchvision.transforms.functional import resize
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from yolov5.models.yolo import Model
from numpy import True
from PIL import Image
from tqdm import tqdm

# BDD100KDataset

In [ ]:
class BDD100KDataset(VisionDataset):
    def __init__(self, labels_file, image_dir, transform=None, target_transform=None, resize_size=(256, 256), crop_size=(224, 224)):
        super(BDD100KDataset, self).__init__(image_dir, transform=transform, target_transform=target_transform)
        self.labels = self._read_labels(labels_file)
        self.images = [item["name"] for item in self.labels]
        self.resize_size = resize_size
        self.crop_size = crop_size

    def __getitem__(self, index):
        image_path = os.path.join(self.root, self.images[index])
        image = Image.open(image_path).convert("RGB")
        image = transforms.Resize(self.resize_size)(image)  #
        image = transforms.RandomCrop(self.crop_size)(image)  # Przekształcenie losowego wycięcia
        label = self.labels[index]
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.labels)

    def _read_labels(self, labels_file):
        with open(labels_file, "r") as f:
            labels = json.load(f)
        return labels

Rezising images to the same resolution.

In [ ]:
def transform(image, target):
    image = image.resize((224, 224))
    image = torch.Tensor(np.array(image)).permute(2, 0, 1) / 255.0  # Konwersja na tensor i normalizacja
    target = torch.Tensor(target)
    return image, target

In [ ]:
# Ścieżki do folderów z obrazami
image_dir_train = '/content/drive/My Drive/car-detection/images/train/train100k'
image_dir_val = '/content/drive/My Drive/car-detection/images/val/'

# Ścieżki do plików etykiet
train_labels_file = '/content/drive/My Drive/car-detection/labels/det_train.json'
val_labels_file = '/content/drive/My Drive/car-detection/labels/det_val.json'

# Tworzenie instancji datasetu treningowego i walidacyjnego
train_dataset = BDD100KDataset(train_labels_file, image_dir_train, transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.ToTensor(),
]))
val_dataset = BDD100KDataset(val_labels_file, image_dir_val, transform=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.ToTensor(),
]))

# Definiowanie DataLoader dla treningu i walidacji
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0)

# Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_memory = True

# Funkcja treningowa
def train(model, train_loader, criterion, optimizer):
    model.train()
    total_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    return avg_loss

In [ ]:
# Inicjalizacja modelu YOLOv5
model = Model(root + "yolov5/models/yolov5s.yaml")

# Załadowanie wstępnie wytrenowanych wag modelu
state_dict = torch.load(root + "yolov5/yolov5s.pt")
model.load_state_dict(state_dict, strict=False)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

# Ustawienie trybu trenowania modelu
model.train()

# Definicja optymalizatora
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Definicja funkcji straty
criterion = torch.nn.CrossEntropyLoss()

# Parametry trenowania
num_epochs = 10
batch_size = 2
learning_rate = 0.001



# Pętla trenowania
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Trening
    train_loss = train(model, train_loader, criterion, optimizer)
    print(f"Train Loss: {train_loss:.4f}")

    # Ocena na zbiorze walidacyjnym (opcjonalnie)
    # val_loss = evaluate(model, val_loader, criterion)
    # print(f"Val Loss: {val_loss:.4f}")

    print("")

# Zapisanie wytrenowanego modelu
torch.save(model.state_dict(), "trained_model.pt")

# Tools

In [ ]:
def split_large_file(input_file, output_folder):
    with open(input_file, 'r') as f:
        data = json.load(f)

    for label in data:
        name = label['name'].replace('.jpg', '.txt')
        output_path = os.path.join(output_folder, name)

        with open(output_path, 'w') as txt_file:
            json.dump(label, txt_file)

        print(f"Zapisano plik: {name}")

# Wprowadź ścieżkę do swojego dużego pliku tekstowego
#input_file = '/content/drive/My Drive/car-detection/labels/det_val_txt.txt'
input_file = '/content/drive/My Drive/car-detection/labels/det_train_txt.txt'

# Określ folder, w którym będą zapisywane podzielone pliki
#output_folder = '/content/drive/My Drive/car-detection/labels/labels_val_txt'
output_folder = '/content/drive/My Drive/car-detection/labels/labels_train_txt'

# Wywołaj funkcję do podziału pliku
split_large_file(input_file, output_folder)

In [ ]:
def convert_to_yolov5(input_folder, output_folder):
    # Słownik mapujący klasy na indeksy w formacie YOLOv5
    class_mapping = {
        "pedestrian": 0,
        "bicycle": 1,
        "car": 2,
        "motorcycle": 3,
        "bus": 5,
        "train": 6,
        "truck": 7,
        "traffic light": 9,
        "traffic sign": 11
        # Brak klasy "rider", bo jest nieistotna
    }

    # Sprawdź istnienie folderu wyjściowego; jeśli nie istnieje, utwórz go
    os.makedirs(output_folder, exist_ok=True)
    count = 0
   # Przetwarzaj każdy plik w folderze wejściowym
    for filename in os.listdir(input_folder):
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, filename)

        with open(input_file, 'r') as f:
            data = json.load(f)
            print("teraz przetwarzam" + str(count) + input_file)

        # Sprawdź, czy klucz 'labels' istnieje w danych
        if 'labels' in data:
            with open(output_file, 'w') as f:
                for label in data['labels']:
                    category = label.get('category', '')
                    if category in class_mapping:
                        class_index = class_mapping[category]

                        x1 = label['box2d'].get('x1', 0)
                        y1 = label['box2d'].get('y1', 0)
                        x2 = label['box2d'].get('x2', 0)
                        y2 = label['box2d'].get('y2', 0)

                        # Oblicz współrzędne środka oraz szerokość i wysokość bounding boxa
                        width = x2 - x1
                        height = y2 - y1
                        x_center = x1 + width / 2
                        y_center = y1 + height / 2

                        # Przeskaluj współrzędne do zakresu 0-1
                        x_center /= 1280
                        y_center /= 720
                        width /= 1280
                        height /= 720

                        # Zapisz wynikowy wiersz w formacie YOLOv5
                        f.write(f"{class_index} {x_center} {y_center} {width} {height}\n")
        else:
            print(f"Brak klucza 'labels' w pliku: {filename}")

        print(f"Przetworzono plik: {filename}")
        count = count +1

# Wprowadź ścieżki do swojego folderu wejściowego i docelowego folderu YOLOv5
#input_folder = "/content/drive/My Drive/car-detection/labels/labels_val_txt"
input_folder = "/content/drive/My Drive/car-detection/labels/labels_train_txt"

#output_folder = "/content/drive/My Drive/car-detection/labels/labels_val_txt_yolov5"
output_folder = "/content/drive/My Drive/car-detection/labels/labels_train_txt_yolov5"

convert_to_yolov5(input_folder, output_folder)

In [ ]:
file_path = "/content/drive/My Drive/car-detection/labels/det_val.json"
output_folder = "/content/drive/My Drive/car-detection/labels/labels_val_txt"

json_file = "/content/drive/My Drive/car-detection/labels/det_val.json"

def create_txt_files(json_file, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for item in data:
        if 'name' in item and 'regions' in item:
            file_name = item['name'].replace('.jpg', '.txt')
            file_path = os.path.join(output_folder, file_name)
            with open(file_path, 'w') as txt_file:
                for region in item['regions']:
                    for line in region.get('lines', []):
                        txt_file.write(line + '\n')
    create_txt_files(file_path, output_folder)

In [ ]:
# Check resolution

def check_resolution(image_folder):
    resolutions = set()

    for filename in os.listdir(image_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(image_folder, filename)
            with Image.open(filepath) as img:
                resolutions.add(img.size)

    if len(resolutions) == 1:
        print("Wszystkie obrazy mają tę samą rozdzielczość:", resolutions.pop())
    else:
        print("Obrazy mają różne rozdzielczości:", resolutions)

image_folder_path = '/content/drive/My Drive/car-detection/images/val'
check_resolution(image_folder_path)

Wszystkie obrazy mają tę samą rozdzielczość: (1280, 720)


In [ ]:
# Show inside of the file

#file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/c79e169f-054f74e7.txt"
#file_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt/b1c66a42-6f7d68ca.txt"
file_path = "/content/drive/My Drive/car-detection/labels/labels_train_txt/11ecaf4a-837e3550.txt"
with open(file_path, 'r') as f:
    content = f.read()

print(content)

{"name": "11ecaf4a-837e3550.jpg", "attributes": {"weather": "clear", "timeofday": "night", "scene": "highway"}, "timestamp": 10000}


In [ ]:
# Json syntax check

def check_json_syntax(folder_path):
    file_names = os.listdir(folder_path)

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)

        with open(file_path, 'r') as f:
            try:
                json.load(f)
                print(f"Plik {file_name} ma poprawną składnię JSON.")
            except json.JSONDecodeError as e:
                print(f"Błąd składni JSON w pliku {file_name}: {e}")

# Wprowadź ścieżkę do folderu zawierającego pliki tekstowe
#folder_path = "/content/drive/My Drive/car-detection/labels/labels_val_txt"
folder_path = "/content/drive/My Drive/car-detection/labels/labels_train_txt"

# Wywołaj funkcję sprawdzającą składnię JSON
check_json_syntax(folder_path)

In [ ]:
# File exists

os.path.exists('/content/drive/My Drive/car-detection/images/train/train100k/8ec22bdd-58cd1abf.jpg')

In [ ]:
# Remove files

def remove_files(folder):
    file_list = os.listdir(folder)
    for file_name in file_list:
        file_path = os.path.join(folder, file_name)
        os.remove(file_path)
        print(f"Usunięto plik: {file_name}")

# Wprowadź ścieżkę do folderu
folder = '/content/drive/My Drive/car-detection/labels/labels_val_txt'

# Wywołaj funkcję do usuwania plików
remove_files(folder)

In [ ]:
# Print labels

image_dir = "/content/drive/My Drive/car-detection/images/train/"
labels_file = "/content/drive/My Drive/car-detection/labels/bdd100k_labels_images_train.json"  # Ścieżka do pliku z etykietami (dostosuj do swojej struktury danych)

# Wczytanie danych etykiet
with open(labels_file, "r") as f:
    labels = json.load(f)

# Sprawdzenie informacji o nazwach plików w etykietach
for item in labels:
    file_name = item["name"]
    # Tutaj możesz wydrukować nazwy plików lub wykonać inne operacje na danych etykiet

# Porównanie nazw plików obrazów z informacjami w etykietach
file_list = os.listdir(image_dir)
for file_name in file_list:
    if file_name not in [item["name"] for item in labels]:
        print("Brak informacji w etykietach dla pliku:", file_name)

In [ ]:
# Print files

image_dir = "/content/drive/My Drive/car-detection/labels/labels_train_txt"

# Wyświetlenie listy plików w folderze
file_list = os.listdir(image_dir)
for file_name in file_list:
    print(file_name)

In [ ]:
# Count files

#!ls '/content/drive/My Drive/car-detection/labels/labels_train_txt' | wc -l
!ls '/content/drive/My Drive/car-detection/labels/labels_train_txt_yolov5' | wc -l
#!ls '/content/drive/My Drive/car-detection/images/train/train100k' | wc -l

In [ ]:
# Move files

src_folder = '/content/drive/My Drive/car-detection'

dst_folder = '/content/drive/My Drive/car-detection'

file_list = glob.glob(os.path.join(src_folder, '*.jpg'))
batch_size = 100  # Liczba plików w jednej partycji
num_batches = len(file_list) // batch_size + 1

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_files = file_list[start_idx:end_idx]

    for file_path in batch_files:
        file_name = os.path.basename(file_path)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.move(file_path, dst_path)
        print(f"Przeniesiono plik: {file_name}")

In [ ]:
print(json_data[0])

In [ ]:
lista_sciezek = set([element["name"] for element in json_data])

# Dalsze prace

## TODO

1.
Przygotowanie danych do treningu
Format json -> yaml
**DONE**

2.
Przejście przez cały tutorial
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#train-on-custom-data


3.
Plan eksperymentów:
?

4.
Ewaluacja pretrenowanego modelu własnych danych BDD100K dataset (próba wyznaczenia prędkości)
Ramka, wyrzucenie szumów (skoki prawo, lewo) regresja liniowa

5.
Wykorzystanie pre-trenowanego modelu na bazie coco128.yaml
Dotrenowanie modelu na ograniczonej ilości klas zgodnie z labelami w coco128.yaml

6.
Wytrenowanie modelu od 0 na BDD100K DATASET (krok opcjonalny)


## TODO_2

1. DOCS postęp pracy

2. Na czym był trenowany yolov5s.pt i ten w tutorialu?


Mam obrazek o pewnej rozdzielczości i prostokąt opisany współrzędnymi. Chcę narysować ramkę, czego użyć?

Mam pre-trenowany model, który rozpoznaje różne klasy obiektów. Zwraca ramkę, ma klasę samochód, mogę zacząc eksperymenty z wyznaczaniem prędkości, a jednocześnie dotrenowywać model z moimi danymi